In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
import time, random, math, string

In [4]:
device = torch.device("mps" if torch.mps.is_available() else "cpu")
#device = torch.device("cpu")

In [11]:
vocab_size = 6336
dim = vocab_size
batch_size = 64
max_len = seq_len = 128
hidden = 2048
decoder = nn.TransformerDecoderLayer(d_model=512, nhead=8).to(device)

memory = torch.rand(1, 1, 512).to(device)
tgt = torch.rand(128, 1, 512).to(device)
out = decoder(tgt, memory)
print(out.shape)  # (128, 64, 6336)

torch.Size([128, 1, 512])


In [ ]:
print(device)
DATA_PATH = "/Users/ibrahimbaldediallo/Documents/Code/Jarvis_project/dataset2.txt"

Préparation des données

In [ ]:
import sentencepiece as spm

# Train SentencePiece model
spm.SentencePieceTrainer.train(input=DATA_PATH, model_prefix='tokenizer',
                               vocab_size=316, model_type='word',
                               unk_piece="<unk>")  # Définit un token pour les mots inconnus


# Load trained tokenizer
sp = spm.SentencePieceProcessor(model_file='tokenizer.model')

# Test tokenization
print(sp.encode("Open Alacritty", out_type=str))  # Example output: ['▁Open', '▁Alacritty']


In [ ]:
pad_token_id = sp.pad_id()  # Récupérer l'ID du token de padding
print(pad_token_id)
print(sp.eos_id())


In [ ]:


# Initialisation des listes et dictionnaires
l = []
X = []
Y = []
word_to_int = {}
int_to_word = {}
current_index = 0

# Lecture du fichier
with open(DATA_PATH, "r") as f:
    for line in f:
        l.append(line.strip())

# Construction des dictionnaires
for data in l:
    split_data = data.split(";")
    x = split_data[0]
    y = split_data[1]

    # Ajouter les mots au dictionnaire
    for word in x.split() + y.split():
        if word not in word_to_int:
            word_to_int[word] = current_index+1
            int_to_word[current_index +1 ] = word
            current_index += 1

    X.append(x)
    Y.append(y)



# Ajout du padding et conversion en tensors
def pad_and_convert_to_tensor(sequences):
    # Convertir les séquences en tensors
    tensor_sequences = [torch.tensor(seq, dtype=torch.int64) for seq in sequences]
    # Ajouter du padding
    padded_sequences = pad_sequence(tensor_sequences, batch_first=True, padding_value=2)
    return padded_sequences

In [ ]:
# Encode dataset
X_encoded = [[sp.bos_id()] + sp.encode(sentence, out_type=int) + [sp.eos_id()] for sentence in X]
Y_encoded = [[sp.bos_id()] + sp.encode(sentence, out_type=int) + [sp.eos_id()] for sentence in Y]



In [ ]:
print(X_encoded[:10])
print(sp.Decode(X_encoded[:10]))
print()
print(Y_encoded[:10])
print(sp.Decode(Y_encoded[:10]))

In [ ]:

X_Tensor = pad_and_convert_to_tensor(X_encoded)
Y_Tensor = pad_and_convert_to_tensor(Y_encoded)

print(X_Tensor.shape)
print(Y_Tensor.shape)

dataset = list(zip(X_Tensor.to(device), Y_Tensor.to(device)))
#dataset = random.sample(dataset, len(dataset))

# Mélange du dataset
random.shuffle(dataset)

def create_batches(dataset, batch_size):
    """
    Divise le dataset en batches de taille batch_size.

    Args:
    - dataset (list): Liste de tuples (X, Y).
    - batch_size (int): Taille de chaque batch.

    Returns:
    - List[List]: Liste de batches.
    """
    batches = [dataset[i:i + batch_size] for i in range(0, len(dataset), batch_size)]
    return batches

# Division du dataset en ensembles d'entraînement et de test
test_dataset = dataset[int(0.8 * len(dataset)):]
train_dataset = dataset[:int(0.8 * len(dataset))]

# Création des batches pour l'ensemble d'entraînement
batch_size = 32
train_batches = create_batches(train_dataset, batch_size)

# Affichage des informations
print(len(train_dataset))
print("Nombre total de batches d'entraînement:", len(train_batches))
print(train_batches[0][0])



In [ ]:
print(train_dataset[0][0])
print()
print(train_batches[0][0][0])

In [ ]:
import fasttext

In [ ]:
# Charger le modèle de FastText (choisir un modèle préentraîné)
ft_model = fasttext.load_model("/Users/ibrahimbaldediallo/Documents/Code/Jarvis_project/cc en 300.bin")  # Modèle anglais

# Créer une matrice d'embeddings avec la même taille que le vocabulaire
embedding_dim = 300
vocab_size = len(sp)  # Taille du vocabulaire de SentencePiece
embedding_matrix = torch.zeros((vocab_size, embedding_dim))

# Remplir la matrice avec les vecteurs des mots connus
for i in range(vocab_size):
    word = sp.id_to_piece(i)
    embedding_matrix[i] = torch.tensor(ft_model.get_word_vector(word))


In [ ]:
print(vocab_size)

In [ ]:
SOS_token = 1
EOS_token = 2
MAX_LENGTH = 17

Création du model

Entrainement du model LSTM

In [ ]:
class AttentionBlock(nn.Module):
    def __init__(self, hidden_size):
        super(AttentionBlock, self).__init__()
        self.attn = nn.Linear(hidden_size * 2, hidden_size)  # Couche linéaire pour les poids
        self.v = nn.Parameter(torch.rand(hidden_size))  # Paramètre appris

    def forward(self, decoder_hidden, encoder_outputs):
        # Appliquer une attention entre l'état du décodeur et chaque état de l'encodeur
        scores = torch.matmul(encoder_outputs, decoder_hidden.unsqueeze(2)).squeeze(2)  # (batch, seq_len)

        # Appliquer un softmax pour obtenir les poids d’attention
        attn_weights = torch.softmax(scores, dim=1)  # (batch, seq_len)

        # Calcul du contexte : somme pondérée des états de l’encodeur
        context_vector = torch.sum(attn_weights.unsqueeze(2) * encoder_outputs, dim=1)  # (batch, hidden_size)

        return context_vector, attn_weights  # (batch_size, hidden_dim), (batch_size, seq_len)


In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding.from_pretrained(embedding_matrix, freeze=False)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, num_layers=n_layers, dropout=dropout)
        
        self.dropout = nn.Dropout(dropout)
    def forward(self, src):
        # src : [sen_len, batch_size]
        embedded = self.dropout(self.embedding(src))
        
        # embedded : [sen_len, batch_size, emb_dim]
        outputs, (hidden, cell) = self.rnn(embedded)
        # outputs = [sen_len, batch_size, hid_dim * n_directions]
        # hidden = [n_layers * n_direction, batch_size, hid_dim]
        # cell = [n_layers * n_direction, batch_size, hid_dim]
        return hidden, cell, outputs

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout, attention):
        super().__init__()
        self.attention = attention
        self.output_dim = output_dim
        self.emb_dim = emb_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding.from_pretrained(embedding_matrix, freeze=False)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, num_layers=self.n_layers, dropout=dropout)
        
        self.fc_out = nn.Linear(2*hid_dim, output_dim)
        self.fc_projection = nn.Linear(521, 2*hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell, encoder_outputs):
        
        # input = [batch_size]
        # hidden = [n_layers * n_dir, batch_size, hid_dim]
        # cell = [n_layers * n_dir, batch_size, hid_dim]
        
        input = input.unsqueeze(0)
        # input : [1, ,batch_size]
        
        embedded = self.dropout(self.embedding(input))
        # embedded = [1, batch_size, emb_dim]
        
        rnn_output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        # output = [seq_len, batch_size, hid_dim * n_dir]
        # hidden = [n_layers * n_dir, batch_size, hid_dim]
        # cell = [n_layers * n_dir, batch_size, hid_dim]

        # Appliquer l’attention de Luong
        context_vector, _ = self.attention(rnn_output.squeeze(0), encoder_outputs)

        #print("rnn_out",rnn_output.shape)
        context_vector = context_vector.mean(dim=1)  # (batch_size, hidden_dim)

        
        cat = torch.cat((rnn_output.squeeze(0), context_vector.unsqueeze(0)), dim=-1)
        cat = self.fc_projection(cat)

        #print("cat" ,cat.shape)

        # Combiner la sortie du RNN et le vecteur de contexte
        prediction = self.fc_out(cat)
        #print("pred", prediction.shape)
        
        # seq_len and n_dir will always be 1 in the decoder
        #prediction = self.fc_out(output.squeeze(0))
        # prediction = [batch_size, output_dim]
        return prediction, hidden, cell

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            'hidden dimensions of encoder and decoder must be equal.'
        assert encoder.n_layers == decoder.n_layers, \
            'n_layers of encoder and decoder must be equal.'
        
    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        src = src.to(self.device)
        trg = trg.to(self.device)
        # src = [sen_len, batch_size]
        # trg = [sen_len, batch_size]
        # teacher_forcing_ratio : the probability to use the teacher forcing.
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        # tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        # last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell, output_encoder = self.encoder(src)
        
        # first input to the decoder is the <sos> token.
        input = trg[0, :]
        for t in range(1, trg_len):
            # insert input token embedding, previous hidden and previous cell states 
            # receive output tensor (predictions) and new hidden and cell states.
            output, hidden, cell = self.decoder(input, hidden, cell,  output_encoder)
            #print("output_encoder",output_encoder.shape)
            # replace predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            # decide if we are going to use teacher forcing or not.
            teacher_force = random.random() < teacher_forcing_ratio
            
            # get the highest predicted token from our predictions.
            top1 = output.argmax(1)
            # update input : use ground_truth when teacher_force 
            input = trg[t] if teacher_force else top1
            
        return outputs

In [ ]:
print(device)

In [ ]:
input_dim = vocab_size
out_dim = vocab_size
emb_dim = 300
hid_dim = 512
n_layers = 4
dropout = 0.3

attention = AttentionBlock(hid_dim)
encoder = Encoder(input_dim, emb_dim, hid_dim, n_layers, dropout).to(device)
decoder = Decoder(out_dim, emb_dim, hid_dim, n_layers, dropout, attention).to(device)
model = Seq2Seq(encoder, decoder, device).to(device)
# xt : [sen_len, batch_size]
xt = train_dataset[0][0]
x_t = xt.unsqueeze(-1)
yt = train_dataset[0][1]
y_t = yt.unsqueeze(-1)
print(yt)
print(sp.decode(xt.tolist()))
print(sp.decode(yt.tolist()))
h, c, o = encoder(x_t)
print(h.shape)
print(c.shape)
z_t = model(x_t.to(device), y_t.to(device))
predicted_indices = z_t.argmax(dim=-1)  # [trg_len, batch_size]
print(z_t.shape)

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

In [ ]:
optimizer = optim.Adam(model.parameters())


criterion = nn.CrossEntropyLoss(ignore_index=2)

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, (X, Y) in enumerate(iterator):
        src = X.unsqueeze(-1)
        trg = Y.unsqueeze(-1)
        
        optimizer.zero_grad()
        # trg = [sen_len, batch_size]
        # output = [trg_len, batch_size, output_dim]
        output = model(src, trg)
        output_dim = output.shape[-1]
        
        # transfrom our output : slice off the first column, and flatten the output into 2 dim.
        output = output[1:].view(-1, output_dim) 
        trg = trg[1:].view(-1)
        # trg = [(trg_len-1) * batch_size]
        # output = [(trg_len-1) * batch_size, output_dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
        
        for i, (X, Y) in enumerate(iterator):
            src = X.unsqueeze(-1)
            trg = Y.unsqueeze(-1)
            
            output = model(src, trg, 0) # turn off teacher forcing.
            
            # trg = [sen_len, batch_size]
            # output = [sen_len, batch_size, output_dim]
            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)
            
            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
            
    return epoch_loss / len(iterator)

In [ ]:
# a function that used to tell us how long an epoch takes.
def epoch_time(start_time, end_time):
    
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time  / 60)
    elapsed_secs = int(elapsed_time -  (elapsed_mins * 60))
    return  elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 4

CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_dataset, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, test_dataset, criterion)
    
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'Seq2SeqModel.pt')
    print(f"Epoch: {epoch+1:02} | Time {epoch_mins}m {epoch_secs}s")
    print(f"\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}")
    print(f"\tValid Loss: {valid_loss:.3f} | Valid PPL: {math.exp(valid_loss):7.3f}")

Test du model LSTM

In [ ]:
model.load_state_dict(torch.load("/Users/ibrahimbaldediallo/Documents/Code/Jarvis_project/Seq2SeqModel.pt", weights_only=True))
model.eval()

In [ ]:
def inference(model, src, max_len, start_token, end_token):
    src = src.to(model.device)
    hidden, cell = model.encoder(src)

    # Initialisation du premier token avec <SOS>
    inputs = torch.tensor([start_token]).to(model.device)

    outputs = []
    
    for _ in range(max_len):
        output, hidden, cell = model.decoder(inputs, hidden, cell)
        token = output.argmax(dim=-1)  # Prédiction du mot avec la plus grande probabilité
        outputs.append(token.item())

        # Arrêter si on atteint le token de fin (<EOS>)
        if token.item() == end_token:
            break

        # Réinjecter la prédiction comme entrée
        input = token.unsqueeze(0)

    return outputs


In [ ]:
def generate_action(sentence, model, max_len, start_token=SOS_token, end_token=6):
    # convert sentence to int then to tensor (with padding)
    x = sp.Encode([sentence], out_type=int)
    xt = pad_and_convert_to_tensor(x) # input size : [1, sentence lenght]
    xt = xt.permute(1, 0) # input size : [sentence lenght, 1]
    print(xt.shape)
    action = inference(model, xt, max_len, start_token, end_token)
    return sp.Decode(action)

In [ ]:
y = inference(model, x_t, 17, 1, 6)
print(x_t.shape)
print(y)
print(sp.Decode(xt.tolist()))
print(sp.Decode(y))

In [ ]:
action = generate_action("Search for something on Google", model, 17)
print(action)